In [1]:
# For Google Colab only
# from google.colab import drive
# drive.mount('/content/drive', True)

Mounted at /content/drive


In [2]:
import sys, os

aima_folder = "/content/drive/MyDrive/BA-CS/COS30019 - Introduction to Artificial Intelligence/aima-python"
os.environ['AIMA_PATH'] = aima_folder
if aima_folder not in sys.path:
    sys.path.append(aima_folder)

In [10]:
import pprint

pprint.pprint(sys.path)
print(os.environ['AIMA_PATH'])

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/BA-CS/COS30019 - Introduction to Artificial '
 'Intelligence/aima-python']
/content/drive/MyDrive/BA-CS/COS30019 - Introduction to Artificial Intelligence/aima-python


In [4]:
# ! pip install -r "$AIMA_PATH/requirements.txt" -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.7/231.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.7 MB/s eta 0:00:00


In [ ]:
# ! pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
asgiref                          3.8.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
b

In [74]:
from typing_extensions import TypeVarTuple
"""
Implement Agents and Environments. (Chapters 1-2)

The class hierarchies are as follows:

Thing ## A physical object that can exist in an environment
    Agent
        Wumpus
    Dirt
    Wall
    ...

Environment ## An environment holds objects, runs simulations
    XYEnvironment
        VacuumEnvironment
        WumpusEnvironment

An agent program is a callable instance, taking percepts and choosing actions
    SimpleReflexAgentProgram
    ...

EnvGUI ## A window with a graphical representation of the Environment

EnvToolbar ## contains buttons for controlling EnvGUI

EnvCanvas ## Canvas to display the environment of an EnvGUI
"""

from utils import distance_squared, turn_heading
from statistics import mean
from ipythonblocks import BlockGrid
from IPython.display import HTML, display, clear_output
from time import sleep

import sys
import random
import copy
import collections
import numbers


# ______________________________________________________________________________


class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass


class Agent(Thing):
    """An Agent is a subclass of Thing with one required instance attribute
    (aka slot), .program, which should hold a function that takes one argument,
    the percept, and returns an action. (What counts as a percept or action
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method, then the
    program could 'cheat' and look at aspects of the agent. It's not supposed
    to do that: the program can only look at the percepts. An agent program
    that needs a model of the world (and of the agent itself) will have to
    build and maintain its own model. There is an optional slot, .performance,
    which is a number giving the performance measure of the agent in its
    environment."""

    def __init__(self, program=None, sensing_radius=1):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        self.sensing_radius = sensing_radius
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        if thing.__class__.__name__ in ['Wall']:
            return False
        return True


def TraceAgent(agent):
    """Wrap the agent's program to print its input and output. This will let
    you see what the agent is doing in the environment."""
    old_program = agent.program

    def new_program(percept):
        action = old_program(percept)
        print('{} perceives {} and does {}'.format(agent, percept, action))
        return action

    agent.program = new_program
    return agent

Agent class is a template for all kinds of agent with a default program: Display the agent's current percept and ask for the action as the input. Example below.

In [ ]:
# ______________________________________________________________________________

agent1=Agent()
print("Is agent1 alive? ", agent1.is_alive())

# ______________________________________________________________________________

Can't find a valid program for Agent, falling back to default.
Is agent1 alive?  True


We will now define some objects(things) that are relevant in the context of the Vacuum Cleaner world

In [75]:
class Obstacle(Thing):
    """Something that can cause a bump, preventing an agent from
    moving into the same square it's in."""
    pass


class Wall(Obstacle):
    pass


class Dirt(Thing):
    pass

class Direction:
    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __add__(self, heading):
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading, None)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading, None)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading, None)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading, None)

    def move_forward(self, from_location):
        iclass = from_location.__class__
        x, y = from_location
        if self.direction == Direction.R:
            return iclass((x + 1, y))
        elif self.direction == Direction.L:
            return iclass((x - 1, y))
        elif self.direction == Direction.U:
            return iclass((x, y - 1))
        elif self.direction == Direction.D:
            return iclass((x, y + 1))

Next, we will define the environments (which are subclasses of the abstract class Environment).

In [76]:
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        # multi-agent, static, fully observable, stochastic, sequential environment
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

The first concrete env is a 2D environment (XYEnvironment)

In [77]:
class XYEnvironment(Environment):
    """This class is for environments on a 2D plane, with locations
    labelled by (x, y) points, either discrete or continuous.

    Agents perceive things within a radius. Each agent in the
    environment has a .location slot which should be a location such
    as (0, 1), and a .holding slot, which should be a list of things
    that are held."""

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width, self.height)

    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """Return all things within radius of location."""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        return [(thing, thing.location)
                for thing in self.things if distance_squared(
                location, thing.location) <= radius2]

    def percept(self, agent):
        """By default, agent perceives things within a default radius."""
        return (agent.location, self.things_near(location=agent.location, radius=agent.sensing_radius))

    def execute_action(self, agent, action):
        print(f"Action: {action}")
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Suck' and 'Dirt' in [thing.__class__.__name__ for thing in self.list_things_at(agent.location)]:
            things = [thing for thing in self.list_things_at(agent.location) if thing.__class__.__name__ == 'Dirt']
            for thing in things:
                self.delete_thing(thing)
        elif action == 'up':
            # The y-coordinate of the agent decreases by 1 (if it's greater than 0, of couse)
            if agent.location[1] > 0:
                new_location = (agent.location[0], agent.location[1] - 1)
                self.move_to(agent, new_location)
        elif action == 'down':
             # The y-coordinate of the agent increases by 1 (if it's less than self.height - 1, of couse)
            if agent.location[1] < self.height - 1:
                new_location = (agent.location[0], agent.location[1] + 1)
                self.move_to(agent, new_location)
        elif action == 'left':
            # The x-coordinate of the agent decreases by 1 (if it's greater than 0, of couse)
            if agent.location[1] > 0:
                new_location = (agent.location[0] - 1, agent.location[1])
                self.move_to(agent, new_location)
        elif action == 'right':
             # The x-coordinate of the agent increases by 1 (if it's less than self.width - 1, of couse)
            if agent.location[1] < self.width - 1:
                new_location = (agent.location[0] + 1, agent.location[1])
                self.move_to(agent, new_location)
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
        elif action == 'Grab':
            things = [thing for thing in self.list_things_at(agent.location) if agent.can_grab(thing)]
            if things:
                agent.holding.append(things[0])
                print("Grabbing ", things[0].__class__.__name__)
                self.delete_thing(things[0])
        elif action == 'Release':
            if agent.holding:
                dropped = agent.holding.pop()
                print("Dropping ", dropped.__class__.__name__)
                self.add_thing(dropped, location=agent.location)
        elif action == 'done':
            agent.alive = False
            print('Agent has finished cleaning')

    def default_location(self, thing):
        location = self.random_location_inbounds()
        while self.some_things_at(location, Obstacle):
            # we will find a random location with no obstacles
            location = self.random_location_inbounds()
        return location

    def move_to(self, thing, destination):
        """Move a thing to a new location. Returns True on success or False if there is an Obstacle.
        If thing is holding anything, they move with him."""
        thing.bump = self.some_things_at(destination, Obstacle)
        if not thing.bump:
            thing.location = destination
            for o in self.observers:
                o.thing_moved(thing)
            for t in thing.holding:
                self.delete_thing(t)
                self.add_thing(t, destination)
                t.location = destination
        return thing.bump

    def add_thing(self, thing, location=None, exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if location is None:
            super().add_thing(thing)
        elif self.is_inbounds(location):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            super().add_thing(thing, location)

    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while location == exclude:
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

    def delete_thing(self, thing):
        """Deletes thing, and everything it is holding (if thing is an agent)"""
        if isinstance(thing, Agent):
            del thing.holding

        super().delete_thing(thing)
        for obs in self.observers:
            obs.thing_deleted(thing)

    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        for x in range(self.width):
            self.add_thing(Wall(), (x, 0))
            self.add_thing(Wall(), (x, self.height - 1))
        for y in range(1, self.height - 1):
            self.add_thing(Wall(), (0, y))
            self.add_thing(Wall(), (self.width - 1, y))
        # print("Added walls. Current world:", self.get_world())

        # Updates iteration start and end (with walls).
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width - 1, self.height - 1)

    def add_observer(self, observer):
        """Adds an observer to the list of observers.
        An observer is typically an EnvGUI.

        Each observer is notified of changes in move_to and add_thing,
        by calling the observer's methods thing_moved(thing)
        and thing_added(thing, loc)."""
        self.observers.append(observer)

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        for x in range(self.width):
            row = []
            for y in range(self.height):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

Next, we define a subclass XYEnvironment (called GraphicEnvironment) that allows the 2D grid to be displayed as square cells and color-coded by the content inside the cell.

In [78]:
class GraphicEnvironment(XYEnvironment):

    basicColor = {"Breeze": (225, 225, 225),
        "Pit": (0,0,0),
        "Gold": (253, 208, 23),
        "Glitter": (253, 208, 23),
        "Wumpus": (43, 27, 23),
        "Stench": (128, 128, 128),
        "Explorer": (0, 0, 255),
        "Wall": (44, 53, 57),
                  # Agent is marked by a bright green
        "Agent":(123, 234, 123),
                  # Dirt is dark green
        "Dirt": (123, 150, 150),
                  # A cell with agent & dirt is marked by red
        "Agent&Dirt": (255, 0, 0)
        }

    def __init__(self, width=10, height=10, boundary=True, color=basicColor, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""
        super().__init__(width, height)

        # The grid has blocks with color gray
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        if self.bounded:
            self.add_walls()
        self.colors = color

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)
        print("Ended")

    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    thing_names = [thing.__class__.__name__ for thing in world[x][y]]
                    if "Agent" in thing_names and "Dirt" in thing_names:
                        self.grid[y, x] = self.colors["Agent&Dirt"]
                    else:
                        self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

Let's now define the first simple reflex Vacuum Cleaner Agent who will detect if there is dirt in its current cell and if there is, it sucks the dirt, if not, it falls back to the default program

In [79]:
def ReflexVacuumAgentGE(sensing_radius=1): # for GraphicEnv
    """
    A reflex agent for the 2D vacuum environment - GraphicEnvironment.
    >>> agent = ReflexVacuumAgentGE()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """

    def program(percept):
        agent_loc, all_things = percept
        all_dirts = [thing for thing in all_things if thing[0].__class__.__name__ == "Dirt"]
        print("the agent's current detectable dirts are: ", all_dirts)

        if all_dirts:
            # check if the agent's current location contains dirt
            all_dirts.sort(key=lambda a: distance_squared(agent_loc, a[1]))
            print("the agent's sorted detectable dirts are: ", all_dirts)
            # if the current location has dirt, the agent will automatically suck it
            if all_dirts[0][1] == agent_loc:
                return 'Suck'
        # return input('Percept={}; action? '.format(percept))
            dirt_x, dirt_y = all_dirts[0][1]
            if dirt_x > agent_loc[0]:
                return 'right'
            elif dirt_x < agent_loc[0]:
                return 'left'
            elif dirt_y > agent_loc[1]:
                return 'down'
            else: # dirt_y < agent_loc[1]
                return 'up'
        return 'done'

    return Agent(program,sensing_radius)

In [91]:
def GoalBasedVacuumAgent(find_path, sensing_radius=1):
    goal = None
    path = []

    def program(percept):
        nonlocal goal, path
        agent_loc, all_things = percept
        all_dirts = [thing for thing in all_things if thing[0].__class__.__name__ == "Dirt"]
        all_obstacles = [thing for thing in all_things if isinstance(thing[0], Obstacle)]
        print("the agent's current detectable dirts are: ", all_dirts)
        print("the agent's current detectable obstacles are: ", all_obstacles)

        # if the has reached its current goal, suck the dirt and reset the goal
        if agent_loc == goal:
            goal = None
            return 'Suck'

        if all_dirts:
            if goal is None:
                max_x, max_y = map(max, zip(*[thing[1] for thing in all_things]))
                agent_map = []
                for y in range(max_y + 1):
                    row = []
                    for x in range(max_x + 1):
                        row.append([])
                    agent_map.append(row)
                for thing in all_things:
                    x, y = thing[1]
                    agent_map[y][x].append(thing[0])
                path = find_path(agent_map, agent_loc)[1:]
                goal = path[-1]
            print("the agent's current goal is: ", goal)
            print("the agent's current path is: ", path)
            next = path.pop(0)
            if next[0] > agent_loc[0]:
                return 'right'
            elif next[0] < agent_loc[0]:
                return 'left'
            elif next[1] > agent_loc[1]:
                return 'down'
            else: # next[1] < agent_loc[1]
                return 'up'
        return 'done'

    return Agent(program,sensing_radius)

In [88]:
def bfs(map, start):
    queue = collections.deque([[start]])
    visited = set([start])
    height = len(map)
    width = len(map[0])
    while queue:
        path = queue.popleft()
        x, y = path[-1]
        at_goal = False
        for thing in map[y][x]:
            if isinstance(thing, Dirt):
                at_goal = True
                break
        if at_goal:
            return path
        for x2, y2 in ((x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)):
            if 0 <= x2 < width and 0 <= y2 < height and (x2, y2) not in visited:
                has_obstacle = False
                for thing in map[y2][x2]:
                    if isinstance(thing, Obstacle):
                        has_obstacle = True
                        break
                if not has_obstacle:
                    queue.append(path + [(x2, y2)])
                    visited.add((x2, y2))

OK, let's try the above agent whose behaviour can be limited. It is not entirely autonomous because its program only does one thing automatically: when the agent is in a square with dirt, it can sense the dirt and immediately sucks the dirt to clean the square. Otherwise, it will need you instructions on what to do next.

In [92]:
# env2 = GraphicEnvironment()
# # env2.add_thing(ReflexVacuumAgentGE(sensing_radius=15),location=(2,2))
# env2.add_thing(GoalBasedVacuumAgent(bfs, sensing_radius=15),location=(2,2))

# env2.add_thing(Dirt(),location=(3,3))
# # env2.add_thing(Dirt(),location=(2,2))
# env2.add_thing(Dirt(),location=(1,1))
# env2.add_thing(Dirt(),location=(6,1))

# env2.add_thing(Wall(),location=(5,1))
# env2.add_thing(Wall(),location=(5,2))
# env2.add_thing(Wall(),location=(5,3))

# # env2.add_thing(Dirt(),location=(2,2))
# # print(env2.get_world())
# # wait = input("Press Enter to continue")
# env2.reveal()
# # env2.get_world()

# env2.run(steps=100)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


the agent's current detectable dirts are:  [(<Dirt>, (6, 1))]
the agent's current detectable obstacles are:  [(<Wall>, (0, 0)), (<Wall>, (0, 9)), (<Wall>, (1, 0)), (<Wall>, (1, 9)), (<Wall>, (2, 0)), (<Wall>, (2, 9)), (<Wall>, (3, 0)), (<Wall>, (3, 9)), (<Wall>, (4, 0)), (<Wall>, (4, 9)), (<Wall>, (5, 0)), (<Wall>, (5, 9)), (<Wall>, (6, 0)), (<Wall>, (6, 9)), (<Wall>, (7, 0)), (<Wall>, (7, 9)), (<Wall>, (8, 0)), (<Wall>, (8, 9)), (<Wall>, (9, 0)), (<Wall>, (9, 9)), (<Wall>, (0, 1)), (<Wall>, (9, 1)), (<Wall>, (0, 2)), (<Wall>, (9, 2)), (<Wall>, (0, 3)), (<Wall>, (9, 3)), (<Wall>, (0, 4)), (<Wall>, (9, 4)), (<Wall>, (0, 5)), (<Wall>, (9, 5)), (<Wall>, (0, 6)), (<Wall>, (9, 6)), (<Wall>, (0, 7)), (<Wall>, (9, 7)), (<Wall>, (0, 8)), (<Wall>, (9, 8)), (<Wall>, (5, 1)), (<Wall>, (5, 2)), (<Wall>, (5, 3))]
the agent's current goal is:  (6, 1)
the agent's current path is:  [(6, 3), (6, 2), (6, 1)]
Action: up


KeyboardInterrupt: 

In [100]:
env = GraphicEnvironment()
env.reveal()

x, y = [int(position.strip()) for position in input('Initial position? (Enter "x, y")\n').split(',')]
location = (x, y)

env.add_thing(ReflexVacuumAgentGE(sensing_radius=15), location=location)
env.reveal()

for i in range(5):
    x, y = [int(position.strip()) for position in input(f'Dirt #{i+1} location? (Enter "x, y")\n').split(',')]
    dirt_location = (x, y)
    env.add_thing(Dirt(),location=dirt_location)
    env.reveal()

wait = input("Press Enter to continue\n")

env.run(steps=100)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Ended


In [ ]:
env = GraphicEnvironment()
has_walls = input("Do you want surrounding walls? (y/n)\n")
if has_walls == "y":
    env.add_walls()
env.reveal()

x, y = [int(position.strip()) for position in input('Initial position? (Enter "x, y")\n').split(',')]
location = (x, y)

env.add_thing(GoalBasedVacuumAgent(bfs, sensing_radius=15),location=location)
env.reveal()

for i in range(5):
    x, y = [int(position.strip()) for position in input(f'Dirt #{i+1} location? (Enter "x, y")\n').split(',')]
    dirt_location = (x, y)
    env.add_thing(Dirt(),location=dirt_location)
    env.reveal()

more_walls = input("Do you want more walls? (y/n)\n").strip().lower()
if more_walls == "y":
    count = eval(input("How many more walls? (Enter an integer number)\n"))
    if type(count) == int:
        for i in range(count):
            x, y = [int(position.strip()) for position in input(f'Wall #{i+1} location? (Enter "x, y")\n').split(',')]
            wall_location = (x, y)
            env.add_thing(Wall(),location=wall_location)
            env.reveal()
    else:
        print("Invalid input. Cancelled adding walls")

wait = input("Press Enter to continue\n")

env.run(steps=100)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Ended
